In [1]:
import re
import sys
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from emoji import emoji_count
import statistics as stat
import nltk
from nltk.corpus import cess_esp

#URLS_RE = re.compile(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b')
URLS_RE = re.compile(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')

LISTING_RE = re.compile(r'^(|[a-z]?|[0-9]{0,3})(\-|\.)+( |\n)')

def remove_urls(text):
    return URLS_RE.sub('', text)

def replace_multi_whitespaces(line):
    return ' '.join(line.split())

def remove_listing(line):
    return LISTING_RE.sub('', line)

def remove_punctuation(text):
    text = text.replace('!','')
    text = text.replace('"','')
    return text.translate(str.maketrans('','',string.punctuation))

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ü","u"),
        ("ñ","n"),
        ("ç","c"),
        ("\u2026","..."),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

def remove_stopwords(text,stop_words):
    words = text.split(' ')
    not_stop_words = []
    for word in words:
        if word not in stop_words:
            not_stop_words.append(word)
    return ' '.join(not_stop_words)
      

def clean_text(text,stop_words):	

    low = text.lower()
    norm = normalize(low)
    rem_u = remove_urls(norm)
    rem_l = remove_listing(rem_u)
    rem_w = replace_multi_whitespaces(rem_l)
    rem_p = remove_punctuation(rem_w)
    rem_s = remove_stopwords(rem_p,stop_words)
    text_enc = rem_s.encode('ascii', 'ignore')

    return text_enc.decode()

def tagger():
    oraciones = cess_esp.tagged_sents()
    oraciones_sin_acentos = []
    for oracion in oraciones:
        oracion_sin_acento = []
        for palabra,tag in oracion:
            palabra = normalize(palabra)
            oracion_sin_acento.append((palabra,tag))
        oraciones_sin_acentos.append(oracion_sin_acento)

    return nltk.UnigramTagger(oraciones_sin_acentos)


def load_embedding(file_path):
    embedding_vectors = {}
    with open(f'embeddings/{file_path}','r') as f:
        first_line = f.readline().split(' ')
        for line in f.readlines()[1:]:
            row = line.split(' ')
            # remove accents
            word = normalize(row[0])
            if word not in embedding_vectors.keys():
                embedding_vectors[word] = [float(val) for val in row[1:]]
        
    return embedding_vectors

def load_sentiment_analysis():
    sent_analysis_data = pd.read_csv('../Spanish-NRC-EmoLex.txt',sep='\t')
    spanish_dict = [normalize(w) for w in list(sent_analysis_data['Spanish Word'])]
    negative_cols = ['negative','fear','anger','disgust','sadness']
    positive_cols = ['positive','joy','trust']

    negative_words = []
    for col in negative_cols:
        i = 0
        for val in sent_analysis_data[col]:
            if val == 1:
                negative_words.append(spanish_dict[i])
            i += 1

    positive_words = []
    for col in positive_cols:
        i = 0
        for val in sent_analysis_data[col]:
            if val == 1:
                positive_words.append(spanish_dict[i])
            i += 1

    return set(positive_words),set(negative_words)

def sent_analysis(words,positive_words,negative_words):

    intersection_neg = list(negative_words & set(words))
    intersection_pos = list(positive_words & set(words))

    return len(intersection_pos),len(intersection_neg)

# returns a list of words that occur exactly 'num' times or None if no coincidence
def num_occurences(lista,num):
    aux_dict = {}
    for item in lista:
        if item in aux_dict.keys():
            aux_dict[item] += 1
        else:
            aux_dict[item] = 1

    try:
        idx = list(aux_dict.values()).index(num)
        words = list(aux_dict.keys())[idx]
        return len(words)
    except:
        return 0


def save_results(y_test,y_pred,average,tunning=False):

    acc = accuracy_score(y_test,y_pred)
    prec = precision_score(y_test,y_pred,average=average)
    rec = recall_score(y_test,y_pred,average=average)
    f1 = f1_score(y_test,y_pred,average=average)

    if tunning:
        sent = '\t* With tunning:\n'
    else:
        sent = '\t* Without tunning:\n'

    with open('results.txt','a') as f:
        f.write(sent)
        f.write(f'\t\t-> Accuracy: {acc}\n')
        f.write(f'\t\t-> Precision: {prec}\n')
        f.write(f'\t\t-> Recall: {rec}\n')
        f.write(f'\t\t-> F1-score: {f1}\n')

In [2]:
# load spanish stop words and remove accents (tweets dont have accents)
stop_words_df = pd.read_csv('../spanish-stop-words.txt',header=None)
stop_words = [normalize(w) for w in list(stop_words_df[0])] + ['q','ma']

data = pd.read_excel('../cleaned_users.xlsx')
username_list = data['username']

print('loading embedding')
embedding_vectors = load_embedding('glove-sbwc.i25.vec')
print('embedding loaded')

loading embedding
embedding loaded


In [12]:
df = pd.DataFrame(columns=[i for i in range(300)])
print('loading sentiment analysis')
positive_words,negative_words = load_sentiment_analysis()
print('sentiment analysis loaded')

punctuation_list = list(string.punctuation)
pos_tag = tagger()

num_mentions = []
num_url = []
num_hashtags = []
num_emojis = []
num_char = []
num_capital = []
num_punctuation = []
num_sentence = []
av_sentence_par = []
av_words_par = []
av_char_par = []
variation = []
num_det = []
num_pre = []
num_sing = []
num_plural = []
num_adv = []
num_adj = []
num_prop = []
num_pronouns = []
num_past = []
num_future = []
num_conj = []
num_words = []
num_pos_words = []
num_neg_words = []
num_unique = []
num_twice = []
av_length = []
max_length = []
num_numbers = []
num_greater = []
num_smaller = []
num_stop = []
for username in username_list[0:1]:
    list_vector_tweet = []

    with open(f'../Documents/{username}.txt','r') as f:
        # twitter features
        doc = f.read()
        num_mentions.append(doc.count('@'))
        num_url.append(doc.count('http://') + doc.count('https://'))
        num_hashtags.append(doc.count('#'))
        num_emojis.append(emoji_count(doc))

    with open(f'../Cleaned Documents/{username}.txt','r') as cf:
        doc = cf.read()
        # character based
        num_char.append(len(doc))
        num_capital.append(sum(1 for c in doc if c.isupper()))
        num_punctuation.append(sum(1 for c in doc if c in punctuation_list))

        # structural based
        doc = doc.lower()
        tweets = doc.split('\n')
        num_par = len(tweets)

        sentences = doc.split('.')
        num_sentence_user = len(sentences)
        num_sentence.append(num_sentence_user)
        av_sentence_par.append(num_sentence_user/num_par)
        
        words = [w for w in doc.split(' ') if len(w) > 0]
        av_words_par.append(len(words)/num_par)
        av_char_par.append(len(doc)/num_par)

        len_sentence_list = [len(sentence) for sentence in sentences]
        variation.append(stat.variance(len_sentence_list))   

        # syntactical based
        # lowercase and remove punctuation marks
        doc = doc.translate(str.maketrans('','',string.punctuation))
        analysis = pos_tag.tag(doc.split(' '))

        det = 0
        pre = 0
        sing = 0
        plural = 0
        adv = 0
        adj = 0
        prop = 0
        pronouns = 0
        past = 0
        future = 0
        conj = 0

        for word,tag in analysis:
            if tag != None:
                if tag[0] == 'd':
                    det += 1
                elif tag[0] == 'a':
                    adj += 1
                elif tag[0] == 'c':
                    conj += 1
                elif tag[0] == 'p':
                    pronouns += 1
                elif tag[0] == 'n':
                    if tag[1] == 'p':
                        prop += 1
                    if tag[3] == 's':
                        sing += 1
                    elif tag[3] == 'p':
                        plural += 1
                elif tag[0] == 'r':
                    adv += 1
                elif (tag[0] == 'v' and tag[3] == 'f'):
                    future += 1
                elif (tag[0] == 'v' and tag[3] == 's'):
                    past += 1
                elif tag[0] == 's':
                    pre += 1

        
        num_det.append(det)
        num_pre.append(pre)
        num_sing.append(sing)
        num_plural.append(plural)
        num_adv.append(adv)
        num_adj.append(adj)
        num_prop.append(prop)
        num_pronouns.append(pronouns)
        num_past.append(past)
        num_future.append(future)
        num_conj.append(conj)

        # word based
        num_words.append(len(words))

        pos, neg = sent_analysis(words,positive_words,negative_words)

        num_pos_words.append(pos)
        num_neg_words.append(neg)

        # unique words
        num_unique.append(num_occurences(words,1))
        # twice occurrences
        num_twice.append(num_occurences(words,2))
        
        # max, av, >6, <3 length and num words with digits, count english words
        max_len = 0
        sum_length = 0
        digits = 0
        len_greater = 0
        len_smaller = 0
        for word in words:
            sum_length += len(word)
            if len(word) > max_len:
                max_len = len(word)

            if len(re.findall('\d',word)) > 0:
                digits += 1

            if len(word) > 6:
                len_greater += 1
            elif len(word) < 3:
                len_smaller += 1
        
        av_length.append(sum_length/len(words))
        max_length.append(max_len)
        num_numbers.append(digits)
        num_greater.append(len_greater)
        num_smaller.append(len_smaller)

        # count stop-words 
        intersection_stop = list(set(stop_words) & set(words))
        num_stop.append(len(intersection_stop))

        # embeddings
        for text in tweets:
            aux_tweet = []
            # clean each tweet
            cleaned = clean_text(text,stop_words)
            # compute vector for each word in the tweet using GloVe
            for token in word_tokenize(cleaned,language='spanish',preserve_line=True):
                vector = embedding_vectors.get(token)
                if vector is not None:
                    aux_tweet.append(vector)
            if len(aux_tweet) != 0:
                vec_tweet = np.asarray(aux_tweet).mean(0)
                list_vector_tweet.append(vec_tweet)
        # compute author's vector averaging tweets vectors
        vec_author = np.asarray(list_vector_tweet).mean(0)
        df.loc[len(df)] = vec_author

gender_dict = {'female':0, 'male':1}

age_dict = {}
i = 0
for age in data['age'].unique():
    age_dict[age] = i
    i += 1

region_dict = {}
i = 0
for region in data['region'].unique():
    region_dict[region] = i
    i += 1
    
df['gender'] = data['gender'].map(gender_dict)
df['age'] = data['age'].map(age_dict)
df['region'] = data['region'].map(region_dict)
df['mentions'] = num_mentions
df['url'] = num_url
df['hashtags'] = num_hashtags
df['emojis'] = num_emojis
df['characters'] = num_char
df['capital_letters'] = num_capital
df['punctuations'] = num_punctuation
df['num_sentence'] = num_sentence
df['av_sentence_par'] = av_sentence_par
df['av_words_par'] = av_words_par
df['av_char_par'] = av_char_par
df['variation'] = variation
df['num_det'] = num_det
df['num_pre'] = num_pre
df['num_sing'] = num_sing
df['num_plural'] = num_plural
df['num_adv'] = num_adv
df['num_adj'] = num_adj
df['num_prop'] = num_prop
df['num_pronouns'] = num_pronouns
df['num_past'] = num_past
df['num_future'] = num_future
df['num_conj'] = num_conj
df['num_words'] = num_words
df['num_pos_words'] = num_pos_words
df['num_neg_words'] = num_neg_words
df['num_unique'] = num_unique
df['num_twice'] = num_twice
df['av_length'] = av_length
df['max_length'] = max_length
df['num_numbers'] = num_numbers
df['num_greater'] = num_greater
df['num_smaller'] = num_smaller
df['num_stop'] = num_stop

loading sentiment analysis
sentiment analysis loaded


In [10]:
features = [i for i in range(300)]
col_drop = [f for f in df.columns if f not in features]
df.drop(col_drop,axis=1,inplace=True)

In [13]:
df.columns

Index([              0,               1,               2,               3,
                     4,               5,               6,               7,
                     8,               9,
       ...
       'num_pos_words', 'num_neg_words',    'num_unique',     'num_twice',
           'av_length',    'max_length',   'num_numbers',   'num_greater',
         'num_smaller',      'num_stop'],
      dtype='object', length=337)

In [2]:
#URLS_RE = re.compile(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b')
URLS_RE = re.compile(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')

LISTING_RE = re.compile(r'^(|[a-z]?|[0-9]{0,3})(\-|\.)+( |\n)')

def remove_urls(text):
    return URLS_RE.sub('', text)

def replace_multi_whitespaces(line):
    return ' '.join(line.split())

def remove_listing(line):
    return LISTING_RE.sub('', line)

def remove_punctuation(text):
    text = text.replace('!','')
    text = text.replace('"','')
    return text.translate(str.maketrans('','',string.punctuation))

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ü","u"),
        ("ñ","n"),
        ("ç","c"),
        ("\u2026","..."),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

def remove_stopwords(text,stop_words):
    words = text.split(' ')
    not_stop_words = []
    for word in words:
        if word not in stop_words:
            not_stop_words.append(word)
    return ' '.join(not_stop_words)
      

def clean_text(text,stop_words):	

    low = text.lower()
    norm = normalize(low)
    rem_u = remove_urls(norm)
    rem_l = remove_listing(rem_u)
    rem_w = replace_multi_whitespaces(rem_l)
    rem_p = remove_punctuation(rem_w)
    rem_s = remove_stopwords(rem_p,stop_words)
    text_enc = rem_s.encode('ascii', 'ignore')

    return text_enc.decode()

In [3]:
embedding_vectors = {}
with open('embeddings/glove-sbwc.i25.vec','r') as f:
    first_line = f.readline().split(' ')
    for line in f.readlines()[1:]:
        row = line.split(' ')
        # remove accents
        word = normalize(row[0])
        if word not in embedding_vectors.keys():
            embedding_vectors[word] = [float(val) for val in row[1:]]
    
    num_words = int(first_line[0])
    emb_dim = int(first_line[1])

print('loaded GloVe')

loaded GloVe


In [48]:
# load spanish stop words and remove accents (tweets dont have accents)
stop_words_df = pd.read_csv('../spanish-stop-words.txt',header=None)
stop_words = [normalize(w) for w in list(stop_words_df[0])] + ['q','ma']

data = pd.read_excel('../cleaned_users.xlsx')
username_list = data['username']

df = pd.DataFrame(columns=[i for i in range(300)])

list_author_vector = []
for username in username_list:
    list_vector_tweet = []
    with open(f'../Cleaned Documents/{username}.txt','r') as f:
        for text in f.readlines():
            aux_tweet = []
            # clean each tweet
            cleaned = clean_text(text,stop_words)
            # compute vector for each word in the tweet using GloVe
            for token in word_tokenize(cleaned,language='spanish',preserve_line=True):
                vector = embedding_vectors.get(token)
                if vector is not None:
                    aux_tweet.append(vector)

            if len(aux_tweet) != 0:
                vec_tweet = np.asarray(aux_tweet).mean(0)
                list_vector_tweet.append(vec_tweet)
        # compute author's vector averaging tweets vectors
        vec_author = np.asarray(list_vector_tweet).mean(0)
        df.loc[len(df)] = vec_author

gender_dict = {'female':0, 'male':1}

age_dict = {}
i = 0
for age in data['age'].unique():
    age_dict[age] = i
    i += 1

region_dict = {}
i = 0
for region in data['region'].unique():
    region_dict[region] = i
    i += 1
    
df['gender'] = data['gender'].map(gender_dict)
df['age'] = data['age'].map(age_dict)
df['region'] = data['region'].map(region_dict)

In [18]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,gender,age,region
0,-0.147999,0.159079,0.013434,0.130502,0.126120,0.045860,0.045618,-0.096360,-0.030039,-0.144021,...,0.042676,-0.007225,-0.035417,-0.032703,-0.015888,0.060023,0.044763,0,0,0
1,0.013895,0.004696,0.104365,-0.012638,0.032255,0.012641,-0.068697,-0.145879,-0.033884,-0.111700,...,0.002509,0.037748,-0.054684,0.039098,0.007610,0.113471,0.015578,1,1,0
2,-0.086022,0.074902,-0.028159,-0.044578,0.119826,0.052359,0.101972,-0.098933,0.060675,0.003302,...,-0.044116,-0.041143,-0.026457,-0.108212,0.135126,0.038695,0.065699,1,1,0
3,-0.152850,0.128347,0.006494,0.050184,0.163639,0.086703,0.029563,-0.111563,-0.031908,-0.006062,...,0.018342,0.008454,-0.054951,-0.107304,0.044327,-0.018754,0.034084,0,1,0
4,-0.063309,0.107178,0.020476,0.089620,0.151430,0.085237,0.009509,-0.140060,-0.034566,-0.064955,...,0.083489,0.031213,-0.020304,-0.082852,-0.005207,0.019539,-0.008659,1,2,0


In [49]:
target = df['region']
df.drop(['gender','age','region'],axis=1,inplace=True)
X = df

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, target, stratify=target, shuffle=True,test_size=0.2,random_state=109)

In [51]:
clf = svm.SVC() 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [52]:
acc = accuracy_score(y_test,y_pred)
prec = precision_score(y_test,y_pred,average='weighted')
rec = recall_score(y_test,y_pred,average='weighted')
f1 = f1_score(y_test,y_pred,average='weighted')

with open('results.txt','a') as f:
    f.write('\t* Without tunning:\n')
    f.write(f'\t\t-> Accuracy: {acc}\n')
    f.write(f'\t\t-> Precision: {prec}\n')
    f.write(f'\t\t-> Recall: {rec}\n')
    f.write(f'\t\t-> F1-score: {f1}\n')

/Users/lunamancebo/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
kernels = list(['linear', 'rbf', 'poly', 'sigmoid'])
c = list([1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 1e2, 1e3, 1e4,1e5])
gammas = list([0.1, 1, 10, 100])

clf = svm.SVC()
clf.fit(X_train, y_train)
param_grid = dict(kernel=kernels, C=c, gamma=gammas)
grid = GridSearchCV(clf, param_grid, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

{'C': 1000.0, 'gamma': 0.1, 'kernel': 'rbf'}

In [54]:
clf = grid.best_estimator_
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test,y_pred)
prec = precision_score(y_test,y_pred,average='weighted')
rec = recall_score(y_test,y_pred,average='weighted')
f1 = f1_score(y_test,y_pred,average='weighted')

with open('results.txt','a') as f:
    f.write('\t* With tunning:\n')
    f.write(f'\t\t-> Accuracy: {acc}\n')
    f.write(f'\t\t-> Precision: {prec}\n')
    f.write(f'\t\t-> Recall: {rec}\n')
    f.write(f'\t\t-> F1-score: {f1}\n')